### Importing libraries

In [4]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torchvision

input_path = './archive/'

In [5]:
normalize = transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])

data_transforms = { # 데이터 전처리 파이프라인을 정의한 딕셔너리입니다. 
    'train' :  # 이 파이프라인은 'train' 데이터와 'validation' 데이터를 위한 전처리 단계를 각각 정의합니다.
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomAffine(0, shear = 10, scale = (0.8, 1.2)), # 이미지에 무작위 어파인 변환(회전, 이동, 크기 조절)을 적용합니다.
        transforms.RandomHorizontalFlip(), # 이미지를 무작위로 수평으로 뒤집습니다.
        transforms.ToTensor(), # 이미지를 PyTorch 텐서 형식으로 변환합니다.
        normalize
    ]),
    'validation' :
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        normalize
    ]),
}

image_datasets = {
    'train' :
    datasets.ImageFolder(input_path + 'training_set/training_set', data_transforms['train']),
    'validation' : 
    datasets.ImageFolder(input_path + 'test_set/test_set', data_transforms['validation'])
}

dataloaders = {
    'train' :
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size = 32,
                                shuffle = True,
                                num_workers = 0),
    'validation' :
    torch.utils.data.DataLoader(image_datasets['validation'],
                                batch_size = 32,
                                shuffle = False,
                                num_workers = 0)

}

## Selecting GPU if available

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [7]:
model = models.resnet50(pretrained=True).to(device)

for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Sequential(
    nn.Linear(2048, 128),
    nn.ReLU(inplace = True),
    nn.Linear(128, 2)).to(device)

/Users/leewonseok/anaconda3/envs/nlp/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/leewonseok/anaconda3/envs/nlp/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /Users/leewonseok/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 64.5MB/s]


In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters())

## Traning

In [11]:
def train_model(model, criterion, optimizer, num_epochs = 3):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, lavels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = lavels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
    return model

In [12]:
model_trained = train_model(model,criterion, optimizer, num_epochs=3)

Epoch 1/3
----------
train loss: 0.1414, acc: 0.9405
Epoch 2/3
----------
train loss: 0.0724, acc: 0.9736
Epoch 3/3
----------


In [ ]:
!mkdir models
!mkdir models/pytorch

In [ ]:
torch.save(model_trained.state_dict(), 'models/pythorch/weights.h5')

### Modifying last Year

In [ ]:
model = models.resnet50(pretrained = False).to(device)
model.fc = nn.Sequential(
    nn.Linear(2048, 128),
    nn.ReLU(inplace = True),
    nn.Linear(128, 2)).to(device)
model.load_state_dict(torch.load('modles/pytorch/weights.h5'))


### Showing some sample images

In [ ]:
validation_img_paths = ['test_set/test_set/cats/cat.4001.jpg',
                        'test_set/test_set/cats/cat.4003.jpg',
                        'test_set/test_set/cats/dog.4004.jpg',
                        'test_set/test_set/cats/dog.4006.jpg']
img_list = [Image.open(input_path + img_path) for img_path in validation_img_paths]